<a href="https://colab.research.google.com/github/irawan09/Click_Prediction/blob/master/GNB_Scikit_Learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Gaussian Naive Bayes

Naive Bayes algorithm is an algorithm that studies the probability of an object with certain characteristics belonging to a certain group/class. In short, it is a probabilistic classifier.

##Importing library

In [70]:
import pandas as pd
import numpy as np
from math import sqrt
from math import pi
from math import exp
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

##Read dataset

In [71]:
train_url = "https://media.githubusercontent.com/media/irawan09/Click_Prediction/master/train_set.csv"
traindataset=pd.read_csv(train_url)
traindataset.head()

,Session Id,DateTime,User Id,Product Type,Campaign Id,Webpage Id,Product Category,Advertisment Size,User Depth,Internet Browser Id,Gender,Age Group,City Size,Device Used,Clicked,Interested In Cars,Interested In Food,Interested In News,Interested In Technology,Interested In Medicine,Interested In Politics,Interested In Fashion,Interested In Astronomy,Interested In Animals,Interested In Travel
0,229ac4c2-0ee9-4a3b-b52c-3b20c9d43039,09-04-2021 00:00,858557,C,359520,13787,4,NaN,3.0,10.0,F,45-54,5900000.0,Mobile,No,1,0,1,0,1,0,0,0,1,0
1,87c0f74a-fa7a-4b3f-bc48-ad1a5f80de2e,09-04-2021 00:00,243253,C,105960,11085,5,NaN,2.0,8.0,F,25-34,NaN,Mobile,No,1,0,1,1,1,0,0,0,1,1
2,b746f0ad-1aa3-492c-b7fa-5dd95643fb51,09-04-2021 00:00,243253,C,359520,13787,4,NaN,2.0,8.0,F,25-34,NaN,Mobile,No,1,0,0,1,0,0,1,0,0,0
3,36257832-77d5-4d8d-9abd-25d49dffb20d,09-04-2021 00:00,1097446,I,359520,13787,3,NaN,3.0,3.0,M,35-44,500000.0,PC/Laptop,No,0,1,1,1,0,0,1,0,0,0
4,df3a66aa-4714-407d-b6fe-49fe03ce8f16,09-04-2021 00:01,663656,C,405490,60305,3,NaN,3.0,2.0,M,25-34,500000.0,PC/Laptop,No,0,1,0,0,0,1,1,1,0,0


In [72]:
test_url = "https://media.githubusercontent.com/media/irawan09/Click_Prediction/master/test_set.csv"
testdataset=pd.read_csv(test_url)
testdataset.head()

,Session Id,DateTime,User Id,Product Type,Campaign Id,Webpage Id,Product Category,Advertisment Size,User Depth,Internet Browser Id,Gender,Age Group,City Size,Device Used,Interested In Cars,Interested In Food,Interested In News,Interested In Technology,Interested In Medicine,Interested In Politics,Interested In Fashion,Interested In Astronomy,Interested In Animals,Interested In Travel
0,41cb91c7-ec1d-4e71-98fd-9e1c374bd33b,14-04-2021 00:00,262150,C,405490,60305,3,NaN,3.0,2.0,M,25-34,500000.0,PC/Laptop,0,1,0,1,0,1,1,1,0,1
1,c9d3f8e4-1445-45f2-99db-754bda50766c,14-04-2021 00:00,308173,C,405490,60305,3,NaN,3.0,10.0,F,45-54,500000.0,PC/Laptop,1,1,1,1,0,1,0,0,1,0
2,f85ffa39-b799-49a1-82b5-2f76a4009212,14-04-2021 00:00,308173,B,118601,28529,4,82527.0,3.0,10.0,F,45-54,500000.0,PC/Laptop,1,0,0,0,1,1,0,1,0,0
3,49336ffb-a9b7-485c-9891-6274482484e0,14-04-2021 00:00,806073,H,118601,28529,5,82527.0,3.0,3.0,M,35-44,500000.0,Mobile,1,0,1,0,1,1,0,1,1,1
4,8ea9c920-6686-46f2-acd1-4bf20371c28c,14-04-2021 00:00,796787,E,404347,53587,1,146115.0,1.0,3.0,M,35-44,NaN,Mobile,1,0,1,0,1,1,1,0,1,0


##Data Preprocesing

Data preprocessing is an initial data mining technique to convert raw data or commonly known as raw data collected from various sources into cleaner information that can be used for further processing. This process can also be called the initial step to retrieve all available information by cleaning, filtering, and combining the data. 3 common problems that are solved in the preprocessing stage are dealing with missing values, noise data, and inconsistent data. Missing value is inaccurate data because missing information makes the information in it irrelevant. Missing value often occurs when there are problems in the collection process, such as errors in data entry or problems in the use of biometrics. Noise data contains erroneous data and outliers that can be found in the data set. These outliers and incorrect data contain meaningless information. Some of the causes of data noise are due to human error in the form of labeling errors and other problems during data collection. Data inconsistency occurs when someone saves a file containing the same data in different formats. Some of the data inconsistencies are duplication in different formats, errors in code names, and so on.

Preprocessing data is very important because errors, redundancy, missing values, and inconsistent data lead to reduced accuracy of analysis results. So, before processing the data, we must ensure that the data we will use is "clean" data.

In [73]:
print (traindataset.groupby('Product Type').groups.keys())
pt_map = {"J" : 9,"I" : 8,"H" : 7,"G" : 6,"F" : 5,"E" : 4,"D" : 3,"C" : 2,"B" : 1, "A" : 0}
traindataset["Product Type"] = traindataset["Product Type"].map(pt_map)

print (traindataset.groupby('Gender').groups.keys())
s_map = {"M" : 1, "F" : 0}
traindataset["Gender"] = traindataset["Gender"].map(s_map)

print (traindataset.groupby('Age Group').groups.keys())
g_map = {">65" : 6, "<18" : 5, "55-64" : 4, "45-54" : 3, "35-44" : 2, "25-34" : 1, "18-24" : 0}
traindataset["Age Group"] = traindataset["Age Group"].map(g_map)


print (traindataset.groupby('Device Used').groups.keys())
du_map = {"Mobile" : 1, "PC/Laptop" : 0}
traindataset["Device Used"] = traindataset["Device Used"].map(du_map)

print (traindataset.groupby('Clicked').groups.keys())
c_map = {"Yes" : 1, "No" : 0}
traindataset["Clicked"] = traindataset["Clicked"].map(c_map)

dict_keys(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'])
dict_keys(['F', 'M'])
dict_keys(['18-24', '25-34', '35-44', '45-54', '55-64', '<18', '>65'])
dict_keys(['Mobile', 'PC/Laptop'])
dict_keys(['No', 'Yes'])


In [74]:
print (testdataset.groupby('Product Type').groups.keys())
pt_map = {"J" : 9,"I" : 8,"H" : 7,"G" : 6,"F" : 5,"E" : 4,"D" : 3,"C" : 2,"B" : 1, "A" : 0}
testdataset["Product Type"] = testdataset["Product Type"].map(pt_map)

print (testdataset.groupby('Gender').groups.keys())
s_map = {"M" : 1, "F" : 0}
testdataset["Gender"] = testdataset["Gender"].map(s_map)

print (testdataset.groupby('Age Group').groups.keys())
g_map = {">65" : 6, "<18" : 5, "55-64" : 4, "45-54" : 3, "35-44" : 2, "25-34" : 1, "18-24" : 0}
testdataset["Age Group"] = testdataset["Age Group"].map(g_map)


print (testdataset.groupby('Device Used').groups.keys())
du_map = {"Mobile" : 1, "PC/Laptop" : 0}
testdataset["Device Used"] = testdataset["Device Used"].map(du_map)

dict_keys(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'])
dict_keys(['F', 'M'])
dict_keys(['18-24', '25-34', '35-44', '45-54', '55-64', '<18', '>65'])
dict_keys(['Mobile', 'PC/Laptop'])


##Deal with missing data

In [75]:
print(traindataset.isnull().sum())

Session Id                       0
DateTime                         0
User Id                          0
Product Type                     0
Campaign Id                      0
Webpage Id                       0
Product Category                 0
Advertisment Size           321591
User Depth                   15743
Internet Browser Id          15743
Gender                       15743
Age Group                    15743
City Size                   107129
Device Used                      0
Clicked                          0
Interested In Cars               0
Interested In Food               0
Interested In News               0
Interested In Technology         0
Interested In Medicine           0
Interested In Politics           0
Interested In Fashion            0
Interested In Astronomy          0
Interested In Animals            0
Interested In Travel             0
dtype: int64


In [76]:
print(testdataset.isnull().sum())

Session Id                      0
DateTime                        0
User Id                         0
Product Type                    0
Campaign Id                     0
Webpage Id                      0
Product Category                0
Advertisment Size           44263
User Depth                   2500
Internet Browser Id          2500
Gender                       2500
Age Group                    2500
City Size                   18000
Device Used                     0
Interested In Cars              0
Interested In Food              0
Interested In News              0
Interested In Technology        0
Interested In Medicine          0
Interested In Politics          0
Interested In Fashion           0
Interested In Astronomy         0
Interested In Animals           0
Interested In Travel            0
dtype: int64


In [77]:
train_dataset = traindataset.drop(['Session Id','Webpage Id','Campaign Id','User Id', 'DateTime', 'Advertisment Size', 'User Depth', 'Internet Browser Id', 'Gender', 'Age Group', 'City Size', ], axis = 1)


In [78]:
test_dataset = testdataset.drop(['Session Id','Webpage Id','Campaign Id','User Id', 'DateTime', 'Advertisment Size', 'User Depth', 'Internet Browser Id', 'Gender', 'Age Group', 'City Size' ], axis = 1)


In [79]:
print(train_dataset.isnull().sum())

Product Type                0
Product Category            0
Device Used                 0
Clicked                     0
Interested In Cars          0
Interested In Food          0
Interested In News          0
Interested In Technology    0
Interested In Medicine      0
Interested In Politics      0
Interested In Fashion       0
Interested In Astronomy     0
Interested In Animals       0
Interested In Travel        0
dtype: int64


In [80]:
print(test_dataset.isnull().sum())

Product Type                0
Product Category            0
Device Used                 0
Interested In Cars          0
Interested In Food          0
Interested In News          0
Interested In Technology    0
Interested In Medicine      0
Interested In Politics      0
Interested In Fashion       0
Interested In Astronomy     0
Interested In Animals       0
Interested In Travel        0
dtype: int64


##Data Training Splitting to evaluate model

Train/test split is one method that can be used to evaluate the performance of machine learning models. This model evaluation method divides the dataset into two parts, namely the part used for training data and for testing data with certain proportions.

Train data is used to fit the machine learning model, while test data is used to evaluate the fit of the model.
Evaluation of machine learning models with train/test splits is suitable for large datasets. As we know, the train/test split divides the dataset into train sets and test sets, or in other words, the data used for the training and testing process are different data sets.

In [81]:
x = train_dataset.drop(['Clicked'], axis = 1)
y = traindataset['Clicked']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 123)

##Constructing and Evaluating The Model

In [82]:
modelnb = GaussianNB()
nbtrain = modelnb.fit(x_train, y_train)

In [83]:
y_pred = nbtrain.predict(x_test)

In [84]:
nbtrain.predict_proba(x_test)

array([[0.9411145 , 0.0588855 ],
       [0.92099313, 0.07900687],
       [0.93343749, 0.06656251],
       ...,
       [0.92185245, 0.07814755],
       [0.93753854, 0.06246146],
       [0.93067429, 0.06932571]])

In [85]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96     73008
           1       0.00      0.00      0.00      5357

    accuracy                           0.93     78365
   macro avg       0.47      0.50      0.48     78365
weighted avg       0.87      0.93      0.90     78365



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Accuracy**
is the ratio of True (positive and negative) predictions to the overall data.

**Precision**
is the ratio of positive true predictions compared to the overall positive predicted outcome.

**Recall (Sensitivity)**
is the ratio of true positive predictions compared to the total number of true positive data.

**Specificity**
It is the correctness of predicting negative compared to the overall negative data.

**F1 Score**
is a weighted comparison of the average precision and recall

##Predicting The Test data File and Construct The Result

In [86]:
print("Number of rows test dataset file :", testdataset[data_test.columns[0]].count())

Number of rows test dataset file : 71466


In [87]:
testdataset.head(3)

,Session Id,DateTime,User Id,Product Type,Campaign Id,Webpage Id,Product Category,Advertisment Size,User Depth,Internet Browser Id,Gender,Age Group,City Size,Device Used,Interested In Cars,Interested In Food,Interested In News,Interested In Technology,Interested In Medicine,Interested In Politics,Interested In Fashion,Interested In Astronomy,Interested In Animals,Interested In Travel
0,41cb91c7-ec1d-4e71-98fd-9e1c374bd33b,14-04-2021 00:00,262150,2,405490,60305,3,NaN,3.0,2.0,1.0,1.0,500000.0,0,0,1,0,1,0,1,1,1,0,1
1,c9d3f8e4-1445-45f2-99db-754bda50766c,14-04-2021 00:00,308173,2,405490,60305,3,NaN,3.0,10.0,0.0,3.0,500000.0,0,1,1,1,1,0,1,0,0,1,0
2,f85ffa39-b799-49a1-82b5-2f76a4009212,14-04-2021 00:00,308173,1,118601,28529,4,82527.0,3.0,10.0,0.0,3.0,500000.0,0,1,0,0,0,1,1,0,1,0,0


In [88]:
new_dataset = traindataset.iloc[:300000]
new_dataset_ = new_dataset.drop(['Clicked'], axis = 1)

In [89]:
print("Number of rows new dataset file :", new_dataset_[new_dataset_.columns[0]].count())

Number of rows new dataset file : 300000


In [90]:
new_dataset_.head(3)

,Session Id,DateTime,User Id,Product Type,Campaign Id,Webpage Id,Product Category,Advertisment Size,User Depth,Internet Browser Id,Gender,Age Group,City Size,Device Used,Interested In Cars,Interested In Food,Interested In News,Interested In Technology,Interested In Medicine,Interested In Politics,Interested In Fashion,Interested In Astronomy,Interested In Animals,Interested In Travel
0,229ac4c2-0ee9-4a3b-b52c-3b20c9d43039,09-04-2021 00:00,858557,2,359520,13787,4,NaN,3.0,10.0,0.0,3.0,5900000.0,1,1,0,1,0,1,0,0,0,1,0
1,87c0f74a-fa7a-4b3f-bc48-ad1a5f80de2e,09-04-2021 00:00,243253,2,105960,11085,5,NaN,2.0,8.0,0.0,1.0,NaN,1,1,0,1,1,1,0,0,0,1,1
2,b746f0ad-1aa3-492c-b7fa-5dd95643fb51,09-04-2021 00:00,243253,2,359520,13787,4,NaN,2.0,8.0,0.0,1.0,NaN,1,1,0,0,1,0,0,1,0,0,0


In [91]:
result = pd.concat([testdataset, new_dataset_])

print("Number of rows both of test dataset :", result[result.columns[0]].count())

Number of rows both of test dataset : 371466


In [92]:
result.head(3)

,Session Id,DateTime,User Id,Product Type,Campaign Id,Webpage Id,Product Category,Advertisment Size,User Depth,Internet Browser Id,Gender,Age Group,City Size,Device Used,Interested In Cars,Interested In Food,Interested In News,Interested In Technology,Interested In Medicine,Interested In Politics,Interested In Fashion,Interested In Astronomy,Interested In Animals,Interested In Travel
0,41cb91c7-ec1d-4e71-98fd-9e1c374bd33b,14-04-2021 00:00,262150,2,405490,60305,3,NaN,3.0,2.0,1.0,1.0,500000.0,0,0,1,0,1,0,1,1,1,0,1
1,c9d3f8e4-1445-45f2-99db-754bda50766c,14-04-2021 00:00,308173,2,405490,60305,3,NaN,3.0,10.0,0.0,3.0,500000.0,0,1,1,1,1,0,1,0,0,1,0
2,f85ffa39-b799-49a1-82b5-2f76a4009212,14-04-2021 00:00,308173,1,118601,28529,4,82527.0,3.0,10.0,0.0,3.0,500000.0,0,1,0,0,0,1,1,0,1,0,0


In [93]:
prob_dataset = result.drop(['Session Id','Webpage Id','Campaign Id','User Id', 'DateTime', 'Advertisment Size', 'User Depth', 'Internet Browser Id', 'Gender', 'Age Group', 'City Size',], axis = 1)

In [94]:
prob_dataset.head(2)

,Product Type,Product Category,Device Used,Interested In Cars,Interested In Food,Interested In News,Interested In Technology,Interested In Medicine,Interested In Politics,Interested In Fashion,Interested In Astronomy,Interested In Animals,Interested In Travel
0,2,3,0,0,1,0,1,0,1,1,1,0,1
1,2,3,0,1,1,1,1,0,1,0,0,1,0


In [95]:
probas = nbtrain.predict_proba(prob_dataset)
prob_list=[]

for i in range(len(probas)):
  maxProbas = np.amax(probas[i])
  prob_list.insert(i, maxProbas)
  
list_of_session_id = result['Session Id'].to_list()
dataset_result = pd.DataFrame(
    {'Session Id': list_of_session_id,
     'Probability': prob_list
    })

dataset_result.head()


,Session Id,Probability
0,41cb91c7-ec1d-4e71-98fd-9e1c374bd33b,0.923655
1,c9d3f8e4-1445-45f2-99db-754bda50766c,0.928902
2,f85ffa39-b799-49a1-82b5-2f76a4009212,0.932182
3,49336ffb-a9b7-485c-9891-6274482484e0,0.942299
4,8ea9c920-6686-46f2-acd1-4bf20371c28c,0.932916


In [96]:
print("Number of rows dataset result :", dataset_result[dataset_result.columns[0]].count())

Number of rows dataset result : 371466


In [97]:
dataset_result.to_csv('result.csv', index=False, encoding='utf-8')